# JSON Toolkit

本 Notebook 展示了一个代理如何与大型 `JSON/dict` 对象进行交互。
当你想回答一个 LLM 上下文窗口无法容纳的 JSON 大块中的问题时，这将非常有用。代理能够迭代地探索该大块以找到回答用户问题所需的信息。

在下面的示例中，我们将使用 OpenAI API 的 OpenAPI 规范，你可以在 [这里](https://github.com/openai/openai-openapi/blob/master/openapi.yaml) 找到它。

我们将使用 JSON 代理来回答有关 API 规范的一些问题。

In [ ]:
%pip install -qU langchain-community

## 初始化

In [2]:
import yaml
from langchain_community.agent_toolkits import JsonToolkit, create_json_agent
from langchain_community.tools.json.tool import JsonSpec
from langchain_openai import OpenAI

In [4]:
with open("openai_openapi.yml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_={}, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=OpenAI(temperature=0), toolkit=json_toolkit, verbose=True
)

## 单个工具

让我们来看看 Jira 工具包中包含哪些单个工具。

In [5]:
[(el.name, el.description) for el in json_toolkit.get_tools()]

[('json_spec_list_keys',
  '\n    Can be used to list all keys at a given path. \n    Before calling this you should be SURE that the path to this exists.\n    The input is a text representation of the path to the dict in Python syntax (e.g. data["key1"][0]["key2"]).\n    '),
 ('json_spec_get_value',
  '\n    Can be used to see value in string format at a given path.\n    Before calling this you should be SURE that the path to this exists.\n    The input is a text representation of the path to the dict in Python syntax (e.g. data["key1"][0]["key2"]).\n    ')]

## 示例：获取请求所需的 POST 参数

In [5]:
json_agent_executor.run(
    "What are the required parameters in the request body to the /completions endpoint?"
)



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the paths key to see what endpoints exist
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the /completions endpoint to see what parameters are required
Action: json_spec_list_keys
Action Input: data["paths"]["/completions"]
Observation: ['post']
Thought: I should look at the post key to see what parameters are required
Actio

"The required parameters in the request body to the /completions endpoint are 'model'."